In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from urllib import request
from scipy.interpolate import interp1d
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import speclite.filters as sp
from speclite import filters
from scipy.ndimage import sum_labels, mean


from astropy.io import fits
from astropy import units as u
from astropy import constants as c
from astropy.wcs import WCS
from astropy.wcs.utils import proj_plane_pixel_scales

from ppxf.ppxf import ppxf, rebin
import ppxf.ppxf_util as util
from ppxf import sps_util as lib

import os
import sys
import glob


In [2]:
# Load spatial binning map IC3392_individual.fits 
# --------- file location (edit if needed) ----------
binning_path = Path("IC3392_SPATIAL_BINNING_maps.fits")
print("Loading:", binning_path.resolve())
with fits.open(binning_path) as hdul:
    # check data structure and header
    print(hdul.info())
    binning_primary = hdul[0]
    binning_BINID   = hdul[1].data
    binning_FLUX    = hdul[2].data
    binning_hdr     = hdul[1].header
    hdul.close()


Loading: /Users/Igniz/Desktop/ICRAR/data/IC3392/IC3392_SPATIAL_BINNING_maps.fits
Filename: IC3392_SPATIAL_BINNING_maps.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  BINID         1 ImageHDU        26   (437, 438)   float64   
  2  FLUX          1 ImageHDU        26   (437, 438)   float64   
  3  SNR           1 ImageHDU        26   (437, 438)   float64   
  4  SNRBIN        1 ImageHDU        26   (437, 438)   float64   
  5  XBIN          1 ImageHDU        26   (437, 438)   float64   
  6  YBIN          1 ImageHDU        26   (437, 438)   float64   
None


In [3]:
# Load SFH and weights data IC3392_sfh-weights.fits
# --------- file location (edit if needed) ----------
sfh_path = Path("IC3392_sfh-weights.fits")
print("Loading:", sfh_path.resolve())
with fits.open(sfh_path) as hdul:
    # check data structure and header
    print(hdul.info())
    weights_data = hdul[1].data
    grid_data = hdul[2].data
    weights_hdr  = hdul[1].header
    grid_hdr  = hdul[2].header

    hdul.close()

Loading: /Users/Igniz/Desktop/ICRAR/data/IC3392/IC3392_sfh-weights.fits
Filename: IC3392_sfh-weights.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      23   ()      
  1  WEIGHTS       1 BinTableHDU     27   4077R x 1C   [477D]   
  2  GRID          1 BinTableHDU     31   477R x 3C   [D, D, D]   
None
None


In [4]:
import numpy as np
from pathlib import Path

# ---------- 2.1  Column names (from HEADER_out_phot) ----------
names = [
    'IMF','slope','MH','Age','U','B','V','R','I','J','H','K',
    'UminusV','BminusV','VminusR','VminusI','VminusJ','VminusH','VminusK',
    'ML_U','ML_B','ML_V','ML_R','ML_I','ML_J','ML_H','ML_K',
    'F439W','F555W','F675W','F814W','C439_555','C555_675','C555_814'
]

fname = Path("BaSTI+Chabrier.dat")

# ---------- 2.2  Load data, skip the two header lines ----------
tbl = np.genfromtxt(
    fname, dtype=None, encoding=None, names=names,
    comments='#', skip_header=2, autostrip=True
)                                  # :contentReference[oaicite:2]{index=2}

# ---------- 2.3  Keep only Chabrier rows ----------
mask = (tbl['IMF'] == 'Ch')
phot = tbl[mask]

print(f"Loaded {len(phot)} out of {len(tbl)} rows of data from {fname.name}")


Loaded 636 out of 636 rows of data from BaSTI+Chabrier.dat


In [5]:
# --- 3.1  Build a lookup dict keyed by (logAge, MH) rounded to 2 dec —--
key_ml = {}

for row in phot:
    age_gyr = round(row['Age'], 2)                     # e.g. 0.03 → 0.03
    mh_dex  = round(row['MH'],  2)                     # e.g. –2.27 → –2.27
    mlr     = row['ML_R']                   # keep 2 dp as requested
    key_ml[(age_gyr, mh_dex)] = mlr


In [6]:

grid = grid_data  # the FITS_rec you already loaded

# --- 4.1  Prepare new array with an extra ML_R column -------------
mlr_values = np.full(len(grid), np.nan, dtype=np.float32)

for i, (logage, mh, _) in enumerate(grid):
    age_gyr = round(10**logage, 2)   # yrs → Gyr, 2 dp
    mh_dex  = round(mh, 2)                 # already dex
    mlr_values[i] = key_ml.get((age_gyr, mh_dex), np.nan)

# --- 4.2  Build a new structured array including ML_R -------------
ml_dtype = grid.dtype.descr + [('ML_R', 'f4')]
grid_mlr  = np.empty(len(grid), dtype=ml_dtype)

for name in grid.dtype.names:
    grid_mlr[name] = grid[name]
grid_mlr['ML_R'] = mlr_values


In [7]:
# --- prerequisites already in memory ---------------------------------------
# weights_data['WEIGHTS']   -> (4077, 477)   light fractions per bin
# grid_mlr['ML_R']          -> (477,)        R-band M/L per template

# 1) convert the opaque FITS_rec into a plain ndarray
w      = weights_data['WEIGHTS'].astype(np.float32)        # (4077, 477)
ml_ssp = grid_mlr['ML_R'].astype(np.float32)               # (477,)

# 2) light-weighted M/L_R per Voronoi bin (shape 4077)
ml_bin = (w * ml_ssp).sum(axis=1)     # or: np.dot(w, ml_ssp)

# 3) optional sanity check: every bin should return a finite, positive value
assert np.all(np.isfinite(ml_bin)) and (ml_bin > 0).all()


In [8]:
# --- 0)  inputs already in memory ------------------------------------------
# binning_BINID  -> (ny, nx) float array   (NaN   = originally masked pixel
#                                            <0    = masked, but *belongs to* |id|)
# ml_bin         -> (N_bin,) float array   (your 4 077 zone M/L_R values)

# --- 1)  create blank map, same shape & dtype ------------------------------
binning_MLR = np.full_like(binning_BINID, np.nan, dtype=np.float32)

# --- 2)  fill *valid* Voronoi zones ----------------------------------------
valid = binning_BINID >= 0                     # True where BINID is a real zone
binning_MLR[valid] = ml_bin[binning_BINID[valid].astype(int)]


In [9]:
# ---------------------------------------------------------------------
# 0.  File paths
# ---------------------------------------------------------------------
cube_path = Path("IC3392_DATACUBE_FINAL_WCS_Pall_mad_red_v3.fits")
vor_path  = Path("IC3392_SPATIAL_BINNING_maps.fits")
sfh_path  = Path("IC3392_SFH_maps.fits")

# ---------------------------------------------------------------------
# 1.  R-band magnitude map per spaxel using speclite
# ---------------------------------------------------------------------
cube = fits.open(cube_path, memmap=False)
data = cube["DATA"].data.astype(np.float32)          # (nz, ny, nx)
hdr  = cube["DATA"].header
nz, ny, nx = data.shape

# wavelength grid (native header unit → Å)
spec_wcs   = WCS(hdr).sub(["spectral"])              # 1-axis WCS
wave_native = spec_wcs.all_pix2world(
                 np.arange(nz)[:, None], 0)[:, 0]    # numeric values
wave = (wave_native * spec_wcs.wcs.cunit[0]).to(u.AA)   # use cunit[0]

# Load filter using speclite (more robust than synphot for this purpose)
f_r = filters.load_filter('bessell-R')  
# decamDR1noatm-r, bessell-R or 'sdss2010noatm-r'/'sdss2010-r' for SDSS r-band

# Convert flux units: MUSE cube is in 1e-20 erg s⁻¹ cm⁻² Å⁻¹
F_lambda = data * (1e-20 * u.erg / (u.s * u.cm**2 * u.AA))  # Add units 

# Calculate AB magnitude for each spaxel
ny, nx = data.shape[1], data.shape[2]
m_r_map = np.full((ny, nx), np.nan, dtype=np.float32)
maggies = np.full((ny, nx), np.nan, dtype=np.float32)

print("Computing AB magnitudes for each spaxel...")
for j in range(ny):
    if j % 50 == 0:  # Progress indicator
        print(f"Processing row {j}/{ny}")
    for i in range(nx):
        # Extract spectrum for this spaxel
        spectrum = F_lambda[:, j, i]
        m_r_map[j, i] = f_r.get_ab_magnitude(spectrum, wavelength=wave)
        maggies[j, i] = f_r.get_ab_maggies(spectrum, wavelength=wave)
cube.close()

# ---------------------------------------------------------------------
# 2.  Collapse to Voronoi bins
# ---------------------------------------------------------------------
with fits.open(vor_path) as hd_v:
    # ---------- PATCH BEGIN ----------
    # keep BINID as float so NaNs survive the read
    BINID_f32 = hd_v["BINID"].data.astype(np.float32)
    muse_hdr2 = hd_v["BINID"].header
    nan_mask  = ~np.isfinite(hd_v["FLUX"].data)

    # build an *integer* copy with sentinel -1 for “no bin”
    bad_pix = (~np.isfinite(BINID_f32)) | (BINID_f32 < 0)
    BINID   = BINID_f32.astype(np.int32)   # safe: all finite →
    BINID[bad_pix] = -1                   # mark blanks with −1
    # ---------- PATCH END ----------

uniq = np.unique(BINID[BINID >= 0])                 # keep this line

# Convert magnitude map to flux for averaging
F0_ref = 3631e-23  # Reference flux in erg s⁻¹ cm⁻² Hz⁻¹ for AB magnitude zero point
flux_map = F0_ref * 10**(-0.4 * m_r_map)  # Convert mag to flux
flux_map[nan_mask] = np.nan  # Keep NaNs where needed

# Average flux in each bin (excluding NaN pixels)
flux_map_clean = np.where(nan_mask | ~np.isfinite(m_r_map), np.nan, flux_map)
valid_pixels = sum_labels(~np.isnan(flux_map_clean), BINID, uniq)
sum_flux = sum_labels(np.nan_to_num(flux_map_clean), BINID, uniq)
mean_flux = np.divide(sum_flux, valid_pixels, 
                     out=np.full_like(sum_flux, np.nan), 
                     where=valid_pixels > 0)

# Convert back to magnitudes
mean_mag = -2.5 * np.log10(mean_flux / F0_ref)

# Create lookup table for bin-averaged magnitudes
lut = np.full(int(BINID.max()) + 1, np.nan, dtype=np.float32)
lut[uniq] = mean_mag
m_r_binned = lut[BINID]                                    # (ny, nx)
m_r_binned[nan_mask] = np.nan  # Keep NaNs where needed
flux_map_binned = F0_ref * 10**(-0.4 * m_r_binned)  # Convert mag to flux

# ---------------------------------------------------------------------
# 3.  Galactic-extinction correction
# ---------------------------------------------------------------------
EBV = fits.getdata(sfh_path, "EBV").astype(np.float32)

# Choose extinction coefficient based on your filter choice:
if 'bessell' in f_r.name.lower():
    A_r = 2.32 * EBV  # Bessell R-band coefficient (Fitzpatrick 1999)
    M_r_sun = 4.64    # Solar absolute magnitude in Bessell R
elif 'sdss' in f_r.name.lower():
    A_r = 2.285 * EBV  # SDSS r-band coefficient (Schlafly & Finkbeiner 2011)
    M_r_sun = 4.64     # Solar absolute magnitude in SDSS r
else:
    print(f"Warning: Unknown filter {f_r.name}, using SDSS coefficients")
    A_r = 2.285 * EBV
    M_r_sun = 4.64

# Apply extinction correction to magnitudes (subtract extinction)
m_r_corr = m_r_binned - A_r  # Magnitude correction
m_r_corr[nan_mask] = np.nan

# magnitude back to nanomaggies in Legacy survey format
def magnitude_to_nanomaggies(magnitude):
    return 10**((22.5 - magnitude) / 2.5)
FLUX_R_corr = magnitude_to_nanomaggies(m_r_corr)  # Convert to flux

# ---------------------------------------------------------------------
# 4.  Luminosity → stellar-mass map
# ---------------------------------------------------------------------
# Distance modulus for 16.5 Mpc
distmod = 5 * np.log10((16.5 * u.Mpc).to(u.pc).value / 10)

# Absolute magnitude
M_r = m_r_corr - distmod

# Luminosity in solar units
L_Lsun = 10**(-0.4 * (M_r - M_r_sun))

# Stellar mass (uncomment when binning_MLR is available)
M_star = L_Lsun * binning_MLR
logM_star = np.where(M_star > 0, np.log10(M_star), np.nan)
logM_star[nan_mask] = np.nan

print("R-band magnitude calculation completed!")
print(f"Filter used: {f_r.name}")
print(f"Magnitude range: {np.nanmin(m_r_corr):.2f} to {np.nanmax(m_r_corr):.2f}")
print(f"Distance modulus: {distmod:.2f} mag")

Computing AB magnitudes for each spaxel...
Processing row 0/438
Processing row 50/438
Processing row 100/438
Processing row 150/438
Processing row 200/438
Processing row 250/438
Processing row 300/438
Processing row 350/438
Processing row 400/438
R-band magnitude calculation completed!
Filter used: bessell-R
Magnitude range: 21.12 to 26.07
Distance modulus: 31.09 mag


/var/folders/rm/clp89v210v32vh5vjt2kx7b80000gr/T/ipykernel_14126/1492270148.py:57: RuntimeWarning: invalid value encountered in cast
  BINID   = BINID_f32.astype(np.int32)   # safe: all finite →


In [10]:
# -------------------------------------------------------------------------
# inputs already in memory:
#   binning_MLR  -> (438, 437) float32   (we’ll cast to float64 like others)
#   binning_path -> Path to original file ("IC3392_SPATIAL_BINNING_maps.fits")
# -------------------------------------------------------------------------

out_path = Path("IC3392_SPATIAL_BINNING_maps_extended.fits")

# 1) read the whole HDUList from disk
with fits.open(binning_path) as hdul:
    # 2) clone all existing HDUs into a new list
    new_hdul = fits.HDUList([hdu.copy() for hdu in hdul])

# 3) build the new FLUX_R_corr image HDU
FLUX_R_corr_hdu = fits.ImageHDU(
    data=FLUX_R_corr.astype(np.float64), name="FLUX_R_corr")

# 4) keep WCS and pixel-scale info by copying the original BINID header
FLUX_R_corr_hdu.header.update(binning_hdr)         # you created binning_hdr earlier
FLUX_R_corr_hdu.header["EXTNAME"] = "FLUX_R_corr"
FLUX_R_corr_hdu.header["BUNIT"]   = "erg/s/cm2/Hz"  # physical units

# 5) append and write to disk
new_hdul.append(FLUX_R_corr_hdu)
new_hdul.writeto(out_path, overwrite=True)

print(f"Saved extended file to {out_path.resolve()}")

Saved extended file to /Users/Igniz/Desktop/ICRAR/data/IC3392/IC3392_SPATIAL_BINNING_maps_extended.fits


In [11]:
with fits.open(out_path, mode="append") as hdul:             # open existing file
    new_hdu = fits.ImageHDU(data=binning_MLR.astype(np.float64),  # like others
                             header=binning_hdr, name="ML_R")
    new_hdu.header["EXTNAME"] = "ML_R"                       # name keyword
    new_hdu.header["BUNIT"] = "Msol/Lsol_R"                   # units keyword
    hdul.append(new_hdu)                                    # add as 9-th HDU
    hdul.flush()                                             # write in-place

print("M/L_R layer saved ➜", out_path.resolve())

M/L_R layer saved ➜ /Users/Igniz/Desktop/ICRAR/data/IC3392/IC3392_SPATIAL_BINNING_maps_extended.fits


In [12]:
# -----------------------------------------------------------------
# INPUTS already in memory
#   logM_star      (438, 437)  float32   ← log10(M*/M☉) per spaxel
#   binning_hdr    WCS header you copied from the BINID image
#   out_path       Path("IC3392_SPATIAL_BINNING_maps_extended.fits")
# -----------------------------------------------------------------

with fits.open(out_path, mode="append") as hdul:             # open existing file
    mass_hdu = fits.ImageHDU(data=logM_star.astype(np.float64),  # like others
                             header=binning_hdr, name="LOGMSTAR")
    mass_hdu.header["BUNIT"] = "log(Msol)"                   # units keyword
    hdul.append(mass_hdu)                                    # add as 9-th HDU
    hdul.flush()                                             # write in-place

print("Stellar-mass layer saved ➜", out_path.resolve())


Stellar-mass layer saved ➜ /Users/Igniz/Desktop/ICRAR/data/IC3392/IC3392_SPATIAL_BINNING_maps_extended.fits


In [13]:
# Getting the stellar mass surface density 
# Convert to surface density in M☉/pc²
# 1. Convert pixel area to physical area in pc²
legacy_wcs2 = WCS(binning_hdr).celestial  # strip spectral axis
pixel_scale = (proj_plane_pixel_scales(legacy_wcs2) * u.deg).to(u.arcsec)
pixel_area_Mpc = ((pixel_scale[0]).to(u.rad).value*16.5*u.Mpc)*(((pixel_scale[1]).to(u.rad).value*16.5*u.Mpc))
pixel_area_kpc = pixel_area_Mpc.to(u.kpc**2)
# 2. Convert stellar mass to surface density
stellar_mass_surface_density = M_star / pixel_area_kpc  # M☉/kpc²
# 3. Convert to log10 scale
log_stellar_mass_surface_density = np.log10(stellar_mass_surface_density.value)

In [14]:
# -----------------------------------------------------------------
# INPUTS already in memory
#   logM_star      (438, 437)  float32   ← log10(M*/M☉) per spaxel
#   binning_hdr    WCS header you copied from the BINID image
#   out_path       Path("IC3392_SPATIAL_BINNING_maps_extended.fits")
# -----------------------------------------------------------------

with fits.open(out_path, mode="append") as hdul:             # open existing file
    mass_density_hdu = fits.ImageHDU(
        data=log_stellar_mass_surface_density.astype(np.float64),  # like others
        header=binning_hdr, name="LOGMASS_SURFACE_DENSITY")
    mass_density_hdu.header["BUNIT"] = "log(Msol/kpc2)"  # units keyword
    hdul.append(mass_density_hdu)                                    # add as 10-th HDU 
    hdul.flush()                                             # write in-place
print("Stellar mass surface density layer saved ➜", out_path.resolve())

Stellar mass surface density layer saved ➜ /Users/Igniz/Desktop/ICRAR/data/IC3392/IC3392_SPATIAL_BINNING_maps_extended.fits


In [15]:
# -----------------------------------------------------------------
# INPUTS already in memory
#   m_R            (438, 437)  float32   ← log10(M*/M☉) per spaxel
#   binning_hdr    WCS header you copied from the BINID image
#   out_path       Path("IC3392_SPATIAL_BINNING_maps_extended.fits")
# -----------------------------------------------------------------

with fits.open(out_path, mode="append") as hdul:             # open existing file
    m_r_hdu = fits.ImageHDU(data=m_r_corr.astype(np.float64),  # like others
                             header=binning_hdr, name="magnitude_r")
    m_r_hdu.header["BUNIT"] = "mag_AB"                   # units keyword
    hdul.append(m_r_hdu)                                    # add as 9-th HDU
    hdul.flush()                                             # write in-place

print("r-band magnitude layer saved ➜", out_path.resolve())

r-band magnitude layer saved ➜ /Users/Igniz/Desktop/ICRAR/data/IC3392/IC3392_SPATIAL_BINNING_maps_extended.fits


In [16]:
# -----------------------------------------------------------------
# INPUTS already in memory
#   m_R            (438, 437)  float32   ← log10(M*/M☉) per spaxel
#   binning_hdr    WCS header you copied from the BINID image
#   out_path       Path("IC3392_SPATIAL_BINNING_maps_extended.fits")
# -----------------------------------------------------------------

with fits.open(out_path, mode="append") as hdul:             # open existing file
    m_r_uncorrected_hdu = fits.ImageHDU(data=m_r_binned.astype(np.float64),  # like others
                             header=binning_hdr, name="magnitude_r_uncorrected")
    m_r_uncorrected_hdu.header["BUNIT"] = "mag_AB"                   # units keyword
    hdul.append(m_r_uncorrected_hdu)                                    # add as 9-th HDU
    hdul.flush()                                             # write in-place
